### Midlevel Vortex Timeseries Plot

Emily Luschen \
Emily.W.Luschen-1@ou.edu \
13 Feb 2025

In [1]:
# import packages
import numpy as np # libary for working with arrays
from grl_mask_tc_track_3d import mask_tc_track_3d # function to mask out the TC track
import grl_functions as fun # custom functions
import xarray as xr
import pickle

Edited: 1 May 2025
Goal: Three panel plot
* Midlevel vortex (600 hPa Vor)

To Do:
1. Read in avor, lat, lon
2. Mask avor
3. Average over lat and lon
4. Pickle vorticity

In [2]:
storm = 'maria' # or 'maria'
ntime = 49 

if storm == 'haiyan':
    wrfout = wrf_path # for lat and lon
    ntest = 4
    tests = ['ncrf36h', 'STRATANVIL_OFF', 'STRATANVIL_ON', 'STRAT_OFF']
    nmem = 8
elif storm == 'maria': # maria
    wrfout = wrf_path
    ntest = 1
    tests = ['ncrf48h'] 
    nmem = 10



# data parameters
p_lev = 16 # only one level available
t0 = 0 # start time index
t1 = 49 # end time index
ntime = 49 # senstivity test time length
membs = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10']

# TC tracking parameters
ptrack='600' # tracking pressure level
var_track = 'rvor' # variable
rmax = 3 # radius (deg) limit for masking around TC center

In [3]:
# 1. Load in lat and lon

varname='XLAT'
lat = fun.var_wrfread(wrfout,varname) # latitude
varname='XLONG'
lon = fun.var_wrfread(wrfout,varname) # latitude

nlat, nlon = np.shape(lat)

### CTL Loop

In [ ]:
# ctl
avor_mean_ctl = np.zeros((nmem, 97))

for m in range(nmem): # member loop
    # 1. Load in data: u,v,avor
    avorfile = avor_path # maria
    df = xr.open_dataset(avorfile)
    avor = df.AVOR.data[:,16,:,:] # time, level, lat, lon
    df.close()

    # 2. Mask out TC track
    track_file = tracking_file # maria
    avor_mask = mask_tc_track_3d(track_file, rmax, avor, lon, lat, t0=0, t1=97)

    # 3. Average over lat and lon and level
    avor_mean_ctl[m,:] = np.ma.mean(avor_mask, axis=(1,2))

    print('Member '+membs[m]+' complete')

# estimated completion time: 17m 47 s
# Maria estimated completion time: 28m 58s (all members)

### Sensitivity Test Loop

In [5]:
# intialize arrays
avor_mean = np.zeros((ntest,nmem,ntime))*np.nan

for itest in range(ntest): # test loop
    print('Now running test: '+tests[itest])
    for m in range(nmem): # member loop
        # 1. Load in data: u,v,avor
        avorfile = avor_path
        df = xr.open_dataset(avorfile)
        avor = df.AVOR.data[:,16,:,:] # time, level, lat, lon
        df.close()

        # 2. Mask out TC track
        track_file = tracking_file
        avor_mask = mask_tc_track_3d(track_file, rmax, avor, lon, lat, t0=0, t1=49) # ncrf track


        # 3. Average over lat and lon and level
        avor_mean[itest,m,:] = np.ma.mean(avor_mask, axis=(1,2))

        print('Member '+membs[m]+' complete')
    print('Test '+tests[itest]+' complete')

    # estimated completion time: 34m 44s

Now running test: ncrf48h
Member 01 complete
Member 02 complete
Member 03 complete
Member 04 complete
Member 05 complete
Member 06 complete
Member 07 complete
Member 08 complete
Member 09 complete
Member 10 complete
Test ncrf48h complete


In [ ]:
# Write out to pickle file
pickle_file = pickle_path
with open(pickle_file, 'wb') as file:
    pickle.dump([avor_mean_ctl, avor_mean], file)
    # avor_mean_ctl: (nmem, time)
    # avor_mean: (ntest, nmem, time)